# referred all from 
[Bike Sharing Demand - Detailed EDA - Periodic Feat](https://www.kaggle.com/code/mexwell/bike-sharing-demand-detailed-eda-periodic-feat)

# table of contents
## 1. data processing
## 2. EDA
### 2-1) distribution - target
### 2-2) distribution - atemp & temp
### 2-3) distribution - datetime
### 2-4 sin cos graph
## 3. rmse

In [177]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from prophet import Prophet
from datetime import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings("ignore")

In [178]:

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

src = '/kaggle/input/bike-sharing-demand'
train = pd.read_csv(src + '/train.csv') 
test = pd.read_csv(src + '/test.csv')

/kaggle/input/bike-sharing-demand/sampleSubmission.csv
/kaggle/input/bike-sharing-demand/train.csv
/kaggle/input/bike-sharing-demand/test.csv
/kaggle/input/bikes-practice-jk/20240224titanic.ipynb
/kaggle/input/bikes-practice-jk/20240303apple-quality-tutorial.ipynb
/kaggle/input/bikes-practice-jk/e_car.ipynb


In [ ]:
train.T

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.describe()

checking on **std**, definately 4columns down below are good to train <br>
since they don't have a big gap
`season`, `holiday`, `working day`, `weather` <br>
std >= 1 `-->` pretty much scattered 

In [ ]:
test

In [ ]:
test.info()

In [ ]:
test.isnull().sum()

In [ ]:
train.shape # (10886, 12)
test.shape # (6493, 9)

In [ ]:
# all = pd.concat([train, test], ignore_index=True, axis = 0)
# all

In [ ]:
train['season'].unique() # array([1, 2, 3, 4])
train['holiday'].unique() # array([0, 1])
train['workingday'].unique() # array([0, 1])
train['weather'].unique() # array([1, 2, 3, 4])
train['temp'].value_counts().head(10)

In [ ]:
train['humidity'].value_counts().head(20)

about bike demands, temp in top 10 is above <br>
i find there's no connection

In [ ]:
train['atemp'].value_counts().head(10)

wind chill in top 10 <br>
( &nbsp;&nbsp; // &nbsp;&nbsp; )

In [ ]:
cat = ['season', 'holiday', 'workingday', 'weather']
for c in cat:
    train[c] = train[c].astype('category')

In [ ]:
season_dict = {1:'Spring', 2:'Summer', 3:'Fall', 4:'Winter'}
weather_dict = {1:'Clear', 2:'Misty+Cloudy', 3:'Light Snow/Rain', 4:'Heavy Snow/Rain'}
train['season'] = train['season'].map(season_dict)
train['weather'] = train['weather'].map(weather_dict)

⬇️let's see if it's got an update

In [ ]:
train['season'].unique()
# Categories (4, object): ['Spring', 'Summer', 'Fall', 'Winter']

train['weather'].unique()
# Categories (4, object): ['Clear', 'Misty+Cloudy', 'Light Snow/Rain', 'Heavy Snow/Rain']

# 2. EDA

#### Distribution - Target <br>
First of all check if we have any outliers in our `target`

In [ ]:
fig = plt.figure(figsize=(20, 5))
sns.boxplot(train['count'])
plt.show();

like you see, full of outliers, lets delete'em by **IQR** 

In [ ]:
def drop_outliers(df, target):
    q1 = df[target].quantile(0.25)
    q3 = df[target].quantile(0.75)
    iqr = q3 - q1
    lower_bound = q1 - 1.5 * iqr
    upper_bound = q3 + 1.5 * iqr
    df_filtered = df[(df[target] >= lower_bound) & (df[target] <= upper_bound)]
    return df_filtered

In [ ]:
train = drop_outliers(train, 'count')

In [ ]:
fig = plt.figure(figsize=(20, 5))
sns.boxplot(train['count'])
plt.show();

strikingly noticeable! you see?

In [ ]:
train.shape
# 10886, 12 --> 10586, 12

great, so we eliminated uneccessary parts

In [ ]:
train['count'].value_counts().head(40)

In [ ]:
train['count'].value_counts().head(40).sum()

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.distplot(train['count'])
plt.show();

Our target is highly skewed to the right. <br>
We have to think about transforming. <br>
Later I will use logsqr since our evaluation metric is NLMSE.

# Distribution - Target vs. Category variables

In [ ]:
group_season = pd.DataFrame(train.groupby(['season'])['count'].mean()).reset_index()
group_holiday = pd.DataFrame(train.groupby(['holiday'])['count'].mean()).reset_index()
group_workingday = pd.DataFrame(train.groupby(['workingday'])['count'].mean()).reset_index()
group_weather = pd.DataFrame(train.groupby(['weather'])['count'].mean()).reset_index()

In [ ]:
fig = plt.figure(figsize=(15, 12))
axes = fig.add_subplot(2, 2, 1)
sns.barplot(data=group_season, x='season', y='count', ax=axes, palette='Set1')
axes.set(xlabel='Season', ylabel='Count', title='Average count across seasons')

axes = fig.add_subplot(2, 2, 2)
sns.barplot(data=group_holiday, x='holiday', y='count', ax=axes, palette='Set1')
axes.set(xlabel='Holiday', ylabel='Count', title='Average count across holidays')

axes = fig.add_subplot(2, 2, 3)
sns.barplot(data=group_workingday, x='workingday', y='count', ax=axes, palette='Set1')
axes.set(xlabel='Workingday', ylabel='Count', title='Average count across workingday')

axes = fig.add_subplot(2, 2, 4)
sns.barplot(data=group_weather, x='weather', y='count', ax=axes, palette='Set1')
axes.set(xlabel='Weather', ylabel='Count', title='Average count across weather conditions')
plt.show();

In [ ]:
weather_data = train[['count', 'temp', 'atemp', 'humidity', 'windspeed']]
sns.pairplot(weather_data, palette='Set1')
plt.show()

there're 2 varaibles --> temp, atemp showing perfect linear pattern,<br>
which means those two parameters are same, so we'd better remove one of'em <br>
since it's giving a tip that there's multicollinearity(다중공선성). <br>
therefore let's draw regression plots(회귀도표) to see linear relationship between those two variables <br><br>


두 변수인 온도(temp)와 체감온도(atemp) 사이에 거의 완벽한 선형 패턴이 있으며, 이는 이러한 변수들 간의 다중공선성(multicollinearity)의 존재를 시사한다는 것을 나타냅니다. 다중공선성은 회귀 분석에서 독립 변수들 간에 강한 상관 관계가 있는 경우에 발생하는 문제로, 이 경우 한 독립 변수가 다른 독립 변수와 상당히 유사하거나 의존적인 경우 발생 <br>

따라서, 두 변수 중 하나를 제거해야 할 필요가 있습니다. 회귀 도표(regression plots)를 빠르게 그려보는 것이 유용하며, 이 도표는 이름에서 알 수 있듯이 두 변수 사이의 회귀선을 생성하고 그들의 선형 관계를 시각화하는 데 도움이 됩니다.

In [ ]:
fig = plt.figure(figsize=(25, 12))
axes = fig.add_subplot(1, 4, 1)
sns.regplot(data=weather_data, x='temp', y='count', ax=axes, scatter_kws={"color": "black"}, line_kws={"color": "red"})
axes = fig.add_subplot(1, 4, 2)
sns.regplot(data=weather_data, x='atemp', y='count', ax=axes, scatter_kws={"color": "black"}, line_kws={"color": "red"})
axes = fig.add_subplot(1, 4, 3)
sns.regplot(data=weather_data, x='humidity', y='count', ax=axes, scatter_kws={"color": "black"}, line_kws={"color": "red"})
axes = fig.add_subplot(1, 4, 4)
sns.regplot(data=weather_data, x='windspeed', y='count', ax=axes, scatter_kws={"color": "black"}, line_kws={"color": "red"})
plt.show();


1. 온도 변수 중 하나를 제거할 수 있으며, 여기서는 atemp를 선택했습니다.
2. 회귀 도표(regplots)는 자전거 대여량(count)과 온도(temp) 및 풍속(windspeed) 간에 양의 상관 관계가 있음을 나타냅니다.
3. 회귀 도표는 자전거 대여량(count)과 습도(humidity) 간에 음의 상관 관계가 있음을 나타냅니다.

1,2,3 요약 --> target인 자전거 대여량(count)은 매우 왜곡되어 있으므로 변형해야 합니다.

# Feature Engineering
### Datetime Features
The column datetime holds a lot of information in it. Let´s extract some datetime features from that column.

In [ ]:
train['datetime'] = pd.to_datetime(train.datetime)
train['hour'] = train.datetime.dt.hour
train['month'] = train.datetime.dt.month
train['year'] = train.datetime.dt.year
train['date'] = train.datetime.dt.date
train['weekday'] = train.datetime.dt.weekday

In [ ]:
f, axes = plt.subplots(1, 1, figsize=(20, 10))
sns.boxplot(data=train, y='count', x='hour', hue='workingday', ax=axes, palette='Set1')
handles, _ = axes.get_legend_handles_labels()
axes.legend(handles, ['Not a Working Day', 'Working Day'])
axes.set(title='Hourly Count based on Working day or not')

plt.show()

You can see a clear pattern. <br>
At night, the demand for bicycles is relatively low. <br>
The slightly higher values when there is no working day are probably due to the fact that people are having a picnic. <br>
On a working day, demand is highest during the typical rush hour. <br>
On the other hand, when it is not a working day, the demand is higher from noon on.

## Monthly Distribution


In [ ]:
f, axes = plt.subplots(nrows=1, ncols=1, figsize=(30, 10))
agg_month = pd.DataFrame(train.groupby(['month', 'workingday'])['count'].mean().reset_index())
sns.barplot(data=agg_month, x='month', y='count', hue='workingday', ax=axes, palette='Set1')
axes.set(xlabel='Month', ylabel='Count', title='Average bike rentals per Month')
handles, _ = axes.get_legend_handles_labels()
axes.legend(handles, ['Not a Working Day', 'Working Day'])
plt.show()

most active rental counts during months consdiered warm weather

## Quarterly Distribution

In [ ]:
f, axes = plt.subplots(nrows=1, ncols=1, figsize=(30, 10))
agg_quarter = pd.DataFrame(train.groupby(['season', 'workingday'])['count'].mean().reset_index())
sns.barplot(data=agg_quarter, x='season', y='count', hue='workingday', ax=axes, palette='Set1')
axes.set(xlabel='season', ylabel='Count', title='Average bike rentals per Quarter')
handles, _ = axes.get_legend_handles_labels()
axes.legend(handles, ['Not a Working Day', 'Working Day'])
plt.show()

- Higher reservations can be seen at around 8am and 5pm (office hours) and close to 0 reservations very early in the morning <br>

- From the above plot we can see the 2 patterns across the hours in a day in bike rentals
- Working Day: First pattern where there is a peak in the rentals at around 8am and another at around 5pm. These correspond to working local bikers who typically are registered and go to work on working day which are Monday to Friday
- Non Working Day: Second pattern where there is more or less a uniform rentals across the day with a peak at around noon time. These correspond to probably tourists who typically are casual users who rent/drop off bikes uniformly during the day and tour the city of Washington on non working days which typically are Saturday and Sunday
- Also, we can see that we have more bike rentals during the Fall (July to September) and Summer (April to June) Season.

# Feature Engineering 
Periodic Features
Periodic features are those that repeat their values at regular intervals, like the hour, the days of a week, and the months of a year.

With cyclical or periodic features, values that are very different in absolute magnitude are actually close. For example, January is close to December, even though their absolute magnitude suggests otherwise.

We can use periodic functions like sine and cosine, to transform cyclical features and help machine learning models pick up their intrinsic nature.

We will apply periodic feature transformation to following features:

- Hour
- Month
- Weekday

In [ ]:
def periodic_transform(df, variable):
    df[f"{variable}_sin"] = np.sin(df[variable] / df[variable].max() * 2 * np.pi)
    df[f"{variable}_cos"] = np.cos(df[variable] / df[variable].max() * 2 * np.pi)
    return df

In [ ]:
train = periodic_transform(train, 'hour')
train = periodic_transform(train, 'month')
train = periodic_transform(train, 'weekday')

In [ ]:
fig = plt.figure(figsize=(15, 15))
axes = fig.add_subplot(3, 2, 1)
sns.scatterplot(data=train, x='hour', y='hour_sin', ax=axes, color='red')
axes = fig.add_subplot(3, 2, 2)
sns.scatterplot(data=train, x='hour', y='hour_cos', ax=axes, color='red')
axes = fig.add_subplot(3, 2, 3)
sns.scatterplot(data=train, x='month', y='month_sin', ax=axes, color='blue')
axes = fig.add_subplot(3, 2, 4)
sns.scatterplot(data=train, x='month', y='month_cos', ax=axes, color='blue')
axes = fig.add_subplot(3, 2, 5)
sns.scatterplot(data=train, x='weekday', y='weekday_sin', ax=axes, color='green')
axes = fig.add_subplot(3, 2, 6)
sns.scatterplot(data=train, x='weekday', y='weekday_cos', ax=axes, color='green')
plt.show();

사인함수, 코사인함수가
변화가 없다가 갑자기 곡선을 그리며 급격하게 바뀐다는 것: <br>
영향을 크게 받았다는 뜻 <br>
쉽게 말해 천천히 내려가다 급격하게 증가 or <br>
천천히 올라가다 급격하게 감소  <br>
ex) 시간 or 계절에 따른 대여량 증가

# Cleanup
Drop columns <br>
Let´s carefully check the dataframe again and identify columns which we can drop.

In [ ]:
train = train.drop(columns=['season',
                            'atemp',
                            'windspeed',
                            'casual',
                            'registered',
                            'datetime',
                            'date',
                            'hour',
                            'month',
                            'weekday'])

## Transform target

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.distplot(train['count'])
plt.show()

we will use `np.log` to transform our target.

### np.log 
데이터의 분포를 정규분포에 더 가깝게 만들 수 있습니다. <br>
이는 모델이 데이터를 더 잘 이해하고 예측할 수 있도록 돕습니다. <br>
로그 변환은 특히 분산이 지나치게 크거나 오른쪽 꼬리가 긴 경우에 유용합니다. <br>
로그 변환을 통해 데이터를 변환하면 데이터의 스케일이 조정되고 이상치의 영향이 줄어들 수 있습니다.

In [ ]:
train['count'] = np.log(train['count'])
fig = plt.figure(figsize=(10, 10))
sns.distplot(train['count'])
plt.show()

# Modelling

In [ ]:
X = train.drop(columns=['count'])
y = train['count']

In [ ]:
def rmsle(y_true, y_pred):
        L1 = np.log1p(np.exp(y_true))
        L2 = np.log1p(np.exp(y_pred))    
        calc = (L1 - L2) ** 2
        return np.sqrt(np.mean(calc))

def print_score(y, pred):
    rmsle_val = rmsle(y, pred)
    print(f'RMSLE: {rmsle_val}')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1702)
X_train.shape, X_val.shape, y_train.shape, y_val.shape

In [ ]:
params = {
    'task': 'train', 
    'boosting': 'gbdt',
    'objective': 'regression',
    'num_leaves': 15,
    'max_depth': 3,
    'learning_rate': 0.1,
    'metric': {'l2','l1'},
    'verbose': -1
}

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train)

In [ ]:
!pip install --upgrade lightgbm

In [ ]:
model = lgb.train(params,
                 num_boost_round=300,
                 train_set=lgb_train,
                 valid_sets=lgb_eval,
#                  early_stopping=20
                 )

In [ ]:
y_pred = model.predict(X_val)
y_pred = np.exp(y_pred)
y_val_exp = np.exp(y_val)

print(f'MAE: {mean_absolute_error(y_val_exp, y_pred)}')
print(f'MSE: {mean_squared_error(y_val_exp, y_pred)}')
print(f'RMSE: {mean_squared_error(y_val_exp, y_pred, squared=False)}')
print_score(y_val_exp, y_pred)

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.scatterplot(x=y_val_exp, y=y_pred)
plt.show();

this linear pattern means that predict values are almost simmilar with actual value

# Hyperparameter Tuning using RandomSearchCV

In [ ]:
param_grid = {
    'boosting_type': ['gbdt', 'dart'],
    'num_leaves': [10, 20, 30],
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [100, 200, 300],
    'subsample': [0.5, 0.8, 1.0],
    'colsample_bytree': [0.5, 0.8, 1.0],
    'reg_alpha': [0.0, 0.1, 0.5],
    'reg_lambda': [0.0, 0.1, 0.5]
}

In [ ]:
regressor = lgb.LGBMRegressor()
random_search = RandomizedSearchCV(estimator=regressor, param_distributions=param_grid,
                                   n_iter=10, scoring='neg_mean_squared_error', cv=5)
random_search.fit(X, y)
print("Best Parameters: ", random_search.best_params_)
print("Best MSE: ", -random_search.best_score_)

In [ ]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_val)

In [ ]:
y_pred = np.exp(y_pred)
y_val_exp = np.exp(y_val)

In [ ]:
print(f'MAE: {mean_absolute_error(y_val_exp, y_pred)}')
print(f'MSE: {mean_squared_error(y_val_exp, y_pred)}')
print(f'RMSE: {mean_squared_error(y_val_exp, y_pred, squared=False)}')
print_score(y_val_exp, y_pred)

In [ ]:
# Plot feature importance
feature_imp = pd.DataFrame(sorted(zip(best_model.feature_importances_,X.columns)), columns=['Value','Feature'])
plt.figure(figsize=(20, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.show();

In [ ]:
fig = plt.figure(figsize=(20, 10))
sns.scatterplot(x=y_val_exp, y=y_pred)
plt.show();

# Predict on testset

Transform testset <br>
We have to apply all transformation we did on the trainset to the testset.

In [ ]:
cats = ['season', 'holiday', 'workingday', 'weather']
for cat in cats:
    test[cat] = test[cat].astype('category')

In [ ]:
test['datetime'] = pd.to_datetime(test.datetime)
test['hour'] = test.datetime.dt.hour
test['month'] = test.datetime.dt.month
test['year'] = test.datetime.dt.year
test['weekday'] = test.datetime.dt.weekday

In [ ]:
sub_datetime = test['datetime']
test = test.drop(columns=['datetime', 
                          'season', 
                          'atemp', 
                          'windspeed', 
                          'datetime', 
                          'hour', 
                          'month', 
                          'weekday'])
test.info()

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'Test shape: {test.shape}')

### Run prediction

In [ ]:
test_preds = best_model.predict(test)
test_preds

### submit

In [ ]:
submission = pd.DataFrame({'datetime': sub_datetime, "count": [max(0, x) for x in np.exp(test_preds)]})
submission

In [ ]:
submission.to_csv('bike-sharing-demand-prediction.csv', index=False)